In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [2]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Search Tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [5]:
from langgraph.prebuilt import ToolNode
from spotify_tools import get_playlists, create_spotify_playlist, get_track_list_from_playlist, add_tracks_to_playlist, filter_artists, get_artists_from_playlist, find_similar_artists, find_top_tracks
from plan import validate_plan

tools = [get_playlists, create_spotify_playlist, add_tracks_to_playlist, filter_artists, validate_plan, get_artists_from_playlist, search_tool, find_similar_artists, get_track_list_from_playlist, find_top_tracks]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Bind Tools to Model

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"), temperature=1.0)
# llm_with_tools = llm.bind_tools(tools, strict=True, parallel_tool_calls=False)
llm_with_tools = llm.bind_tools(tools, strict=True)

# System Prompt

In [7]:
from langchain_core.messages import SystemMessage
from prompts import Prompts

system_message = SystemMessage(Prompts.SYSTEM)


# First Message

In [8]:
from langchain_core.messages import HumanMessage
from prompts import Prompts

human_message = HumanMessage(Prompts.SPOTIFY)
llm_response = llm_with_tools.invoke([system_message, human_message])

2024-11-14 12:55:22,734 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [9]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "validate_plan":
    raise ValueError("should be tool message calling 'validate_plan'")

In [10]:
llm_response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  validate_plan (call_SoqGUCt51YuwHyp5vhw8GMnp)
 Call ID: call_SoqGUCt51YuwHyp5vhw8GMnp
  Args:
    plan: {'steps': [{'step_number': 1, 'name': 'retrieve_original_artists', 'type': 'action', 'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.", 'success_criteria': "A list of unique artist names and URIs from the 'New Rock and Blues' playlist is obtained.", 'tool': 'functions.get_artists_from_playlist', 'action': "Call get_artists_from_playlist with the playlist ID of 'New Rock and Blues'.", 'condition': None, 'substeps': None}, {'step_number': 2, 'name': 'find_similar_artists', 'type': 'loop', 'description': 'For each artist retrieved in the previous step, find 3-4 similar artists.', 'success_criteria': 'A list of similar artists for each original artist is obtained.', 'tool': 'functions.find_similar_artists', 'action': None, 'condition': 'for each artist

In [11]:
plan = llm_response.tool_calls[0]["args"]
plan

{'plan': {'steps': [{'step_number': 1,
    'name': 'retrieve_original_artists',
    'type': 'action',
    'description': "Retrieve the list of unique artists from the 'New Rock and Blues' playlist.",
    'success_criteria': "A list of unique artist names and URIs from the 'New Rock and Blues' playlist is obtained.",
    'tool': 'functions.get_artists_from_playlist',
    'action': "Call get_artists_from_playlist with the playlist ID of 'New Rock and Blues'.",
    'condition': None,
    'substeps': None},
   {'step_number': 2,
    'name': 'find_similar_artists',
    'type': 'loop',
    'description': 'For each artist retrieved in the previous step, find 3-4 similar artists.',
    'success_criteria': 'A list of similar artists for each original artist is obtained.',
    'tool': 'functions.find_similar_artists',
    'action': None,
    'condition': 'for each artist in the list of original artists',
    'substeps': [{'step_number': 1,
      'name': 'get_similar',
      'type': 'action',
   

# chat_prompt_template holds all messages (Human, AI, Tool)

In [12]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt_template: ChatPromptTemplate = system_message + human_message + llm_response
messages = chat_prompt_template.format_messages()
messages

[SystemMessage(content="\n    You are an AI language assistant designed to execute tasks with precision and attention to detail. Your primary objective is to follow the user's instructions thoroughly and exactly as specified, leaving no detail unaddressed.\n\n    Guidelines:\n\n    - **Carefully Read Instructions:** Before starting any task, read all user instructions thoroughly to ensure complete understanding.\n\n    - **Create a Detailed Plan:** Develop a step-by-step plan that addresses every aspect of the user's requirements. Validate this plan before execution.\n\n    - **Utilize Advanced Planning Structures:** Do not limit yourself to linear or sequential plans. Employ loops for repetitive tasks, branches for conditional logic, and other programming constructs to handle complex tasks and decision-making processes.\n\n    - **Follow Instructions Precisely:** Execute each step exactly as described, without omitting or altering any part of the instructions.\n\n    - **Be Thorough:*

# Check for Tools in Message

In [ ]:
from langchain_core.messages import ToolMessage

tools_in_ai_message = []
tools_by_name = {tool.name: tool for tool in tools}
messages = chat_prompt_template.format_messages()
for tool_call in messages[-1].tool_calls:
    tool = tools_by_name[tool_call["name"]]
    tools_in_ai_message.append(tool)
    # observation = tool.invoke(tool_call["args"])
    # result.append(ToolMessage(content=observation, tool_call_id=tool_call["id"]))
if len(tools_in_ai_message) == 0:
    raise ValueError
else:
    print(tools_in_ai_message)

# Tool Call (should be validate_plan())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
tool_message = tool_response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
chat_prompt_template.format_messages()

# Send Messages with Tool result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(chat_prompt_template.format_messages())
llm_response.pretty_print()

# Validate correct tool call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_playlists":
    raise ValueError("should be tool message calling 'validate_plan'")

# Tool Call (should be get_playlists())

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
tool_response = tool_node.invoke({"messages": messages})

In [ ]:
from langchain_core.messages import ToolMessage

tool_message: ToolMessage = tool_response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()

# Send Playlists Results to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

# Validate correct tool call

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "get_artists_from_playlist":
    raise ValueError("should be tool message calling 'get_artists_from_playlist'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()

# Tool Call (get_artists_from_playlist())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
tool_message: ToolMessage = tool_response["messages"][0]

# Decode AI Tool Call message

In [ ]:
tool_message.pretty_print()

In [ ]:
# json.loads(track_list_tool_message.content)

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)

# AI Tool Call (find_similar_artist())

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'validate_plan'")

In [ ]:
llm_response.pretty_print()

In [ ]:
if llm_response.tool_calls is None or llm_response.tool_calls[0]["name"] != "find_similar_artists":
    raise ValueError("should be tool message calling 'find_similar_artists'")

# Add AI Tool Call to Messages

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

# Tool Call (find_similar_artists())

In [ ]:
tool_response = tool_node.invoke({"messages": messages})
tool_message = tool_response["messages"][0]
tool_message.pretty_print()

# Add Tool Response to Messages

In [ ]:
chat_prompt_template += tool_message
messages = chat_prompt_template.format_messages()
messages

# Send Tool Result to LLM

In [ ]:
llm_response = llm_with_tools.invoke(messages)
llm_response.pretty_print()

In [ ]:
chat_prompt_template += llm_response
messages = chat_prompt_template.format_messages()
messages

In [ ]:
filtered_artists: ToolMessage = response["messages"][0]
filtered_artists.pretty_print()

In [ ]:
response = llm_with_tools.invoke(messages)

In [ ]:
chat_prompt_template += response
chat_prompt_template.format_messages()

In [ ]:
response = tool_node.invoke({"messages": [response]})

In [ ]:
response